In [1]:
import pandas as pd
import numpy as np
from tasks import perc_diff
import uuid
import argparse
import os
from datetime import datetime

from tasks import (
    load_config,
    load_dataset_iscx,
    get_quantity,
    get_days,
    get_counts_days,
    get_counts_hours,
    get_counts_hours_sub_mean_benign,
    check_filters
)
from filter import (
    filter_all_days_stats,
    filter_perc_count
)

# add flag option for donfig json file
parser = argparse.ArgumentParser()
parser.add_argument("-c", "--config", required=True, help="Path to config file", dest="config")
args = parser.parse_args()
config = args.config

config = load_config(config)

dataset_path = config['dataset']
dataset_config = load_config(f'{dataset_path}config.json')

data, benign_days = load_dataset_iscx(dataset_path=dataset_path, inputs=dataset_config['inputs'],
                                      inputs_benign=dataset_config['inputs_benign'], headers=dataset_config['headers'])
quantity = get_quantity(data=data, columns=["rule_name", "sid"])

days = get_days(data=data)
if check_filters(config['filters'], dataset_config['filters']):

    for filter in config['filters']:
        if filter == 'all_days':
            group_cols = dataset_config['filters'][filter]['group_cols']
            stats = get_counts_days(data=data, group_cols=group_cols)
        elif filter == 'count_hours':
            group_cols = dataset_config['filters'][filter]['group_cols']
            stats = get_counts_hours(data=data, group_cols=group_cols)
        elif filter == 'count_hours_except_first_day':
            group_cols = dataset_config['filters'][filter]['group_cols']
            except_first_day = sorted(days, key=lambda x: datetime.strptime(x, '%Y-%m-%d'))[1:]
            stats = get_counts_hours_sub_mean_benign(data, group_cols=group_cols, benign_days=benign_days)
            break
        stats = stats.reset_index()
        filtered_stats = filter_perc_count(stats, dataset_config['filters'][filter])
        # print(filtered_stats)

    # stats.to_csv('to_delete.csv')
    # print(stats)

    # print(filter_all_days_stats(data=stats_all_days_resind, conf=dataset_config['filters']['all_days']))

    # FILTER
    # print(stats_all_days)
    # print(data)
    
stats


usage: ipykernel_launcher.py [-h] -c CONFIG
ipykernel_launcher.py: error: the following arguments are required: -c/--config


SystemExit: 2

/home/ferik/.local/share/virtualenvs/tomas-bajtos-projekt-R9kYtVgK/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
